In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import datetime
import os
import sys
import calendar
from tqdm.notebook import tqdm
from collections import deque
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import pickle
import yaml
from dotmap import DotMap
import neptune.new as neptune

In [3]:
import utils

In [4]:
import sys
sys.path.append("../common")
import common_utils

In [5]:
CONFIG_PATH = "./config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = DotMap(yaml.safe_load(f), _dynamic=False)

In [6]:
config.pprint(pformat="dict")

{'data': {'first': {'month': 11, 'year': 2020},
          'last': {'month': 12, 'year': 2020},
          'symbol': 'usdjpy'},
 'feature': {'freqs': ['1min', '5min', '15min', '1h', '4h'],
             'lag_max': 10,
             'sma': {'timing': 'close', 'window_size': 10},
             'timings': ['high', 'low', 'close']},
 'gcp': {'bucket_name': 'preprocessed-thashimoto',
         'project_id': 'auto-trader-359210',
         'secret_id': 'neptune_api_key'},
 'label': {'thresh_entry': 0.05, 'thresh_exit': 0.0, 'thresh_hold': 0.025},
 'model': {'objective': 'binary', 'random_state': 123},
 'neptune': {'model_key': 'LGBM',
             'project': 'thashimoto/sandbox',
             'project_key': 'SAN'},
 'on_colab': False,
 'train': {'num_iterations': 20, 'save_model': True, 'test_ratio': 0.1}}


In [8]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../auto-trader-sa.json"
os.environ["NEPTUNE_PROJECT"] = config.neptune.project

In [9]:
secretmanager = common_utils.SecretManagerWrapper(config.gcp.project_id)
neptune_api_token = secretmanager.fetch_secret(config.gcp.secret_id)
os.environ["NEPTUNE_API_TOKEN"] = neptune_api_token

In [16]:
run = neptune.init_run(run="SAN-7")

https://app.neptune.ai/thashimoto/sandbox/e/SAN-7
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [12]:
run["config"].fetch()

{'data': {'first': {'month': 11, 'year': 2020},
  'last': {'month': 12, 'year': 2020},
  'symbol': 'usdjpy'},
 'feature': {'freqs': "['1min', '5min', '15min', '1h', '4h']",
  'lag_max': 10,
  'sma': {'timing': 'close', 'window_size': 10},
  'timings': "['high', 'low', 'close']"},
 'gcp': {'bucket_name': 'preprocessed-thashimoto',
  'project_id': 'auto-trader-359210',
  'secret_id': 'neptune_api_key'},
 'label': {'thresh_entry': 0.05, 'thresh_exit': 0.0, 'thresh_hold': 0.025},
 'model': {'objective': 'binary', 'random_state': 123},
 'neptune': {'model_key': 'LGBM',
  'project': 'thashimoto/sandbox',
  'project_key': 'SAN'},
 'on_colab': False,
 'train': {'num_iterations': 20, 'save_model': True, 'test_ratio': 0.1}}

In [18]:
type(run["retrain/long_entry/importance"].fetch())

str

In [19]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/thashimoto/sandbox/e/SAN-7
